# Week 2: Identify Nearest Health Facilities

<span style="color:red">
**UPDATE**

Thank you for your analysis. Despite our warning efforts so far, the virus continues to spread rapidly. We want to get infected individuals treatment as quickly as possible, so we need your help to calculate which hospital or clinic is closest to each known infected individual in the population.
</span>

Your goal for this notebook will be to identify the nearest hospital or clinic for each infected person.

## Imports

In [3]:
import cudf
import cuml
import cupy as cp

## Load Population Data

Begin by loading the `lat`, `long` and `infected` columns from `'./data/week2.csv'` into a cuDF data frame called `gdf`.

In [4]:
# 加载人口数据
file_path = './data/week2.csv'
gdf = cudf.read_csv(file_path, usecols=['lat', 'long', 'infected'])

## Load Hospital and Clinics Data

For this step, your goal is to create an `all_med` cuDF data frame that contains the latitudes and longitudes of all the hospitals (data found at `'./data/hospitals.csv'`) and clinics (data found at `'./data/clinics.csv'`).

In [6]:
# 加载医院和诊所数据
hospitals_df = cudf.read_csv('./data/hospitals.csv')
clinics_df = cudf.read_csv('./data/clinics.csv')

Since we will be using the coordinates of those facilities, keep only those rows that are non-null in both  `Latitude` and `Longitude`.

In [7]:

# 合并医院和诊所数据
all_med = cudf.concat([hospitals_df, clinics_df]).reset_index(drop=True)

# 删除纬度和经度为空的行
all_med = all_med.dropna(subset=['Latitude', 'Longitude']).reset_index(drop=True)

## Make Grid Coordinates for Medical Facilities

Provided for you in the next cell (which you can expand by clicking on the "...", and contract again after executing by clicking on the blue left border of the cell) is the lat/long to grid coordinates converter you have used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `all_med` data frame you created in the last step.

In [12]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [13]:

# 转换医院和诊所的经纬度到网格坐标
all_med['northing'], all_med['easting'] = latlong2osgbgrid_cupy(all_med['Latitude'].to_cupy(), all_med['Longitude'].to_cupy())

## Find Closest Hospital or Clinic for Infected

Fit `cuml.NearestNeighbors` with `all_med`'s `northing` and `easting` values, using the named argument `n_neighbors` set to `1`, and save the model as `knn`.

In [15]:
# 使用cuml.NearestNeighbors找到最近的医院或诊所
import cudf
import cuml
from cuml.neighbors import NearestNeighbors
import cupy as cp
knn = NearestNeighbors(n_neighbors=1)
knn.fit(all_med[['northing', 'easting']])

NearestNeighbors()

Save every infected member in `gdf` into a new dataframe called `infected_gdf`.

In [16]:
# 过滤出感染者数据
infected_gdf = gdf[gdf['infected'] == 1].reset_index(drop=True)

Create `northing` and `easting` values for `infected_gdf`.

In [17]:
# 转换感染者的经纬度到网格坐标
infected_gdf['northing'], infected_gdf['easting'] = latlong2osgbgrid_cupy(infected_gdf['lat'].to_cupy(), infected_gdf['long'].to_cupy())

Use `knn.kneighbors` with `n_neighbors=1` on `infected_gdf`'s `northing` and `easting` values. Save the return values in `distances` and `indices`.

In [18]:
# 找到最近的医院或诊所
distances, indices = knn.kneighbors(infected_gdf[['northing', 'easting']], n_neighbors=1)

In [19]:
# 展示结果
print("感染者坐标：")
print(infected_gdf[['lat', 'long', 'northing', 'easting']].head())

print("\n最近的医院或诊所索引：")
print(indices.head())



感染者坐标：
         lat      long       northing        easting
0  53.715826 -2.430079  424489.783814  371619.678741
1  53.664881 -2.425673  418820.687944  371876.492369
2  53.696765 -2.488940  422394.398940  367721.000265
3  53.696966 -2.488897  422416.821887  367723.973098
4  53.727804 -2.392959  425808.109929  374076.557677

最近的医院或诊所索引：
0    18316
1    12816
2     7785
3     7785
4     4962
dtype: int64


### Check Your Solution

`indices`, returned from your use of `knn.kneighbors` immediately above, should map person indices to their closest clinic/hospital indices:

In [20]:
indices.head()

0    18316
1    12816
2     7785
3     7785
4     4962
dtype: int64

Here you can print an infected individual's coordinates from `infected_gdf`:

In [21]:
infected_gdf.iloc[0] # get the coords of an infected individual (in this case, individual 0)

lat             53.715826
long            -2.430079
infected         1.000000
northing    424489.783814
easting     371619.678741
Name: 0, dtype: float64

You should be able to used the mapped index for the nearest facility to see that indeed the nearest facility is at a nearby coordinate:

In [22]:
all_med.iloc[1234] # printing the entry for facility 1234 (replace with the index identified as closest to the individual)

,﻿OrganisationID,OrganisationCode,OrganisationType,SubType,Sector,OrganisationStatus,IsPimsManaged,OrganisationName,Address1,Address2,...,Latitude,Longitude,ParentODSCode,ParentName,Phone,Email,Website,Fax,northing,easting
1234,17942,NAA06,Clinic,UNKNOWN,<NA>,Visible,False,Manor Farm Health Centre,Manor Farm Road,<NA>,...,53.404556,-2.831035,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,390099.13988,344757.527843


<div align="center"><h2>Please Restart the Kernel</h2></div>

...before moving to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)